In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from datasets import load_dataset, Dataset

# flower
import flwr as fl
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner

c:\Users\niloy\miniconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_file = 'attention_numeric.csv'
df = pd.read_csv(data_file)


train_df = df[:3200]
test_df = df[3200:]

train_ds = Dataset.from_pandas(train_df, split='train')
test_ds = Dataset.from_pandas(test_df, split='test')

# main_dataset = load_dataset("csv", data_files=data_file)
# main_dataset

In [8]:
cols = train_ds.column_names

In [16]:
train_ds.features

{'no_of_face': Value(dtype='int64', id=None),
 'face_x': Value(dtype='float64', id=None),
 'face_y': Value(dtype='float64', id=None),
 'face_w': Value(dtype='float64', id=None),
 'face_h': Value(dtype='float64', id=None),
 'face_con': Value(dtype='float64', id=None),
 'no_of_hand': Value(dtype='int64', id=None),
 'pose': Value(dtype='int64', id=None),
 'pose_x': Value(dtype='float64', id=None),
 'pose_y': Value(dtype='float64', id=None),
 'phone': Value(dtype='int64', id=None),
 'phone_x': Value(dtype='int64', id=None),
 'phone_y': Value(dtype='int64', id=None),
 'phone_w': Value(dtype='int64', id=None),
 'phone_h': Value(dtype='int64', id=None),
 'phone_con': Value(dtype='float64', id=None),
 'label': Value(dtype='int64', id=None)}

# Partitioner

In [17]:
partitioner = IidPartitioner(num_partitions=10)

partitioner.dataset = train_ds

In [18]:
partitioner.load_partition(partition_id=0)


Dataset({
    features: ['no_of_face', 'face_x', 'face_y', 'face_w', 'face_h', 'face_con', 'no_of_hand', 'pose', 'pose_x', 'pose_y', 'phone', 'phone_x', 'phone_y', 'phone_w', 'phone_h', 'phone_con', 'label'],
    num_rows: 320
})

In [19]:
NUM_CLIENTS = 10
BATCH_SIZE = 32

trainloaders = []
valloaders = []
for partition_id in range(NUM_CLIENTS):
    partition = partitioner.load_partition(partition_id)
    
    partition = partition.train_test_split(train_size=0.8, seed=42)
    trainloaders.append(DataLoader(partition["train"], batch_size=BATCH_SIZE))
    valloaders.append(DataLoader(partition["test"], batch_size=BATCH_SIZE))

testloader = DataLoader(test_ds, batch_size=BATCH_SIZE)



In [59]:
next(iter(trainloaders[0]))

{'no_of_face': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 1, 1, 1, 1, 1, 1, 1]),
 'face_x': tensor([203.4054, 270.3198, 280.1867, 325.8162, 267.7397, 302.9323, 178.5900,
         343.6002, 257.8106, 305.7684, 251.1568, 178.7871, 180.6091, 307.8918,
         221.7130, 198.0118, 314.1122, 312.4948, 326.6386, 306.9357, 341.2012,
         345.9195, 327.8390, 215.0534,   0.0000, 310.2180, 273.5754, 251.9702,
         179.9767, 205.0999, 303.3341, 332.0424], dtype=torch.float64),
 'face_y': tensor([195.3048, 274.9657, 239.2570, 369.4451, 114.9439, 278.4474, 217.9682,
         269.0542, 224.3967, 277.0767, 275.3308, 124.4607, 222.5235, 276.6091,
         241.2522, 181.9824, 277.5124, 277.5345, 367.4226, 225.9265, 357.5038,
         363.5866, 353.3090, 224.9158,   0.0000, 298.3477, 254.5960, 204.2641,
         223.2516, 229.2937, 285.7629, 285.2076], dtype=torch.float64),
 'face_w': tensor([138.5814, 164.1431, 157.8290, 169.1203, 238.8236, 173.5

In [42]:
features = {
    'no_of_face': batch['no_of_face'],
    'face_x': batch['face_x'],
    'face_y': batch['face_y'],
    'face_w': batch['face_w'],
    'face_h': batch['face_h'],
    'face_con': batch['face_con'],
    'no_of_hand': batch['no_of_hand'],
    'pose': batch['pose'],
    'pose_x': batch['pose_x'],
    'pose_y': batch['pose_y'],
    'phone': batch['phone'],
    'phone_x': batch['phone_x'],
    'phone_y': batch['phone_y'],
    'phone_w': batch['phone_w'],
    'phone_h': batch['phone_h'],
    'phone_con': batch['phone_con']
}

labels = batch['label']

# # Example of processing the features and labels
# print("Features:")
# for key, value in features.items():
#     print(f"{key}: {value}")

# print("Labels:")
# print(labels)


# pytorch model

In [62]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [84]:
# define model
class attentionNet(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.hidden_1 = nn.Linear(16, 80)
        self.act_1 = nn.ReLU()
        
        self.hidden_2 = nn.Linear(80, 400)
        self.act_2 = nn.ReLU()
        
        self.hidden_3 = nn.Linear(400, 80)
        self.act_3 = nn.ReLU()
        
        self.hidden_4 = nn.Linear(80,16)
        self.act_4 = nn.ReLU()
        
        self.out = nn.Linear(16, 1)
        self.act_out = nn.Sigmoid()
        
    def forward(self, x):
        x = self.act_1(self.hidden_1(x))
        x = self.act_2(self.hidden_2(x))
        x = self.act_3(self.hidden_3(x))
        x = self.act_4(self.hidden_4(x))
        x = self.act_out(self.out(x))
        
        return x

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define the training function
def train(model, train_loader, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for inputs, labels in next(iter(trainloaders[0])):
            
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# Define the testing function
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            preds = outputs.round()
            correct += (preds == labels).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')
